## インストール

In [140]:
# pip install pingouin
# pip install plotnine
# pip install mysql-connector-python
# pip install numexpr
# pip install python-dotenv
# pip install japanize-matplotlib




SyntaxError: invalid syntax (<ipython-input-140-17a64f5d11c8>, line 8)

In [143]:
pip install xlwt


     |████████████████████████████████| 99 kB 2.3 MB/s eta 0:00:01
You should consider upgrading via the '/Users/oxyu8/.pyenv/versions/3.8.5/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from itertools import combinations

import csv
import MySQLdb
import mysql.connector as mydb
import pandas as pd
import pandas.io.sql as psql
import numpy as np
from pingouin import kruskal, mwu, multicomp
from plotnine import *


In [78]:
# %load_ext dotenv
# %dotenv

%reload_ext dotenv


In [2]:
host = os.environ.get('HOST')
user = os.environ.get('USER_NAME')
password = os.environ.get('PASSWORD')
database = os.environ.get('DATABASE')
question_database = os.environ.get('QUESTION_UI_DATABASE')
DB_CONFIG = dict(host=host, user=user, password=password, database=database)
QUESTION_DB_CONFIG = dict(
    host=host, user=user, password=password, database=question_database)


def check_db_connection(db_config=QUESTION_DB_CONFIG):
  connection = mydb.connect(**db_config)
  connection.ping(reconnect=True)
  # 接続できているかどうか確認
  print(connection.is_connected())


check_db_connection(QUESTION_DB_CONFIG)


True


In [3]:
UI_CONDITION = ("question", "perspective", "non")

THEME_CONDITION = ("tpp", "airPollution", "vietnamTrip")

TPP_UI_CONDITION = (
    "TPP_WITH_QUERYUI",
    "TPP_WITH_NONUI",
)
AIRPOLLUTION_UI_CONDITION = (
    "AIRPOLLUTION_WITH_QUERYUI",
    "AIRPOLLUTION_WITH_NONUI",
)
VIETNAMTRIP_UI_CONDITION = (
    "VIETNAMTRIP_WITH_QUERYUI",
    "VIETNAMTRIP_WITH_NONUI"
)


In [4]:

def get_dataframe_from_db(sql, db_config):
  """
  db上のデータをデータフレームとして取得
  """
  conn = mydb.connect(**db_config)
  df_read = pd.read_sql(sql, conn)
  conn.close()
  return df_read


def convert_csv_to_array(csv_path):
  """
  csvファイルの中身を一次元配列へ変換
  """
  result_arr = []
  with open(csv_path) as f:
      reader = csv.reader(f)
      for row in reader:
        result_arr.append(str(row[0]))
  return result_arr


def convert_singleArray_to_dataframe(array, columnName):
  df = pd.DataFrame(array, columns=[columnName],)
  return df


## 分析対象とする実験協力者

- ウェブ検索タスクにて意見文章の提出まで行い，事後アンケートへの回答が完了しているワーカー
- タスク所要時間が外れ値ではないワーカー

In [5]:

# ウェブ検索タスクと事後アンケートへの回答の両方が完了しているワーカー
questionnaire_answered_crowdIds_csv_path = './data/questionnaire_answered_crowdId.csv'
questionnaire_answered_crowdIds = convert_csv_to_array(questionnaire_answered_crowdIds_csv_path)
questionnaire_answered_crowdIds_df = convert_singleArray_to_dataframe(
    questionnaire_answered_crowdIds, 'crowdId')
users_who_complete_webSearchTask_df = get_dataframe_from_db("select * from users where hasFinishedWebSearchTask", DB_CONFIG)
users_who_completed_all_task_df =  users_who_complete_webSearchTask_df[users_who_complete_webSearchTask_df['crowdId'].isin(questionnaire_answered_crowdIds)]

all_task_done_user_crowdIds = users_who_completed_all_task_df['crowdId'].tolist()

# タスク所要時間の外れ値処理
def get_dataframe_without_outlier(df, target_column):
    q1 = df[target_column].quantile(
        0.25)
    q3 = df[target_column].quantile(
        0.75)
    iqr = q3 - q1
    threshold_upper = q3 + 1.5 * iqr
    threshold_lower = q1 - 1.5 * iqr
    df["threshold_lower"] = threshold_lower
    df["threshold_upper"] = threshold_upper

    df_without_outlier = df[(df[target_column] < threshold_upper) & (
        df[target_column] > threshold_lower)]
    return df_without_outlier

task_time_df = get_dataframe_from_db("select * from taskTime;", DB_CONFIG)
task_time_with_users_done_all_task_df = task_time_df[task_time_df['crowdId'].isin(
    all_task_done_user_crowdIds)]
task_time_with_users_done_all_task_df["taskTimeDuration"] = (
    task_time_with_users_done_all_task_df["finished_at"]-task_time_with_users_done_all_task_df["reStarted_at"]).map(lambda x: x.total_seconds())

valid_users_df = get_dataframe_without_outlier(
    task_time_with_users_done_all_task_df, 'taskTimeDuration')
valid_users_df.head()
valid_user_crowdIds = valid_users_df.crowdId.tolist()
print(len(valid_user_crowdIds))
# print(len(valid_users_df)) # 264人

## 問いかけ群ワーカー
# ウェブ検索タスクと事後アンケートへの回答の両方が完了している問いかけ群ワーカー
questionnaire_answered_questionUI_crowdIds_csv_path = './data/questionnaire_answered_questionUI_crowdId.csv'
questionnaire_answered_questionUI_crowdIds = convert_csv_to_array(
    questionnaire_answered_questionUI_crowdIds_csv_path)
questionnaire_answered_questionUI_crowdIds_df = convert_singleArray_to_dataframe(
    questionnaire_answered_questionUI_crowdIds, 'crowdId')
q_users_df = get_dataframe_from_db(
    "select * from users where hasFinishedWebSearchTask;", QUESTION_DB_CONFIG)
q_users_who_completed_all_task_df = q_users_df[q_users_df['crowdId'].isin(
    questionnaire_answered_questionUI_crowdIds)]
q_all_task_done_user_crowdIds = q_users_who_completed_all_task_df['crowdId'].tolist()

q_task_time_df = get_dataframe_from_db('select * from taskTime;', QUESTION_DB_CONFIG)
q_task_time_with_users_done_all_task_df = q_task_time_df[q_task_time_df['crowdId'].isin(
    q_all_task_done_user_crowdIds)]
q_task_time_with_users_done_all_task_df["taskTimeDuration"] = (
    q_task_time_with_users_done_all_task_df["finished_at"]-q_task_time_with_users_done_all_task_df["reStarted_at"]).map(lambda x: x.total_seconds())
q_valid_users_df = get_dataframe_without_outlier(
    q_task_time_with_users_done_all_task_df, 'taskTimeDuration')
q_valid_user_crowdIds = q_valid_users_df.crowdId.tolist()
print(len(q_valid_user_crowdIds)) # 87人




264
87


<ipython-input-4-afed83266c14>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
<ipython-input-4-afed83266c14>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
<ipython-input-5-c74a205bad19>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<ipython-input-5-c74a205bad19>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

## 実験協力者の内訳

- questionUI群に関しては，別データで分析を行うため，perspectiveUI，nonUIのみ集計する

In [6]:
THEME_UI_CONDITION = (
                        "TPP_WITH_QUERYUI",
                        "AIRPOLLUTION_WITH_QUERYUI",
                        "VIETNAMTRIP_WITH_QUERYUI",
                        "TPP_WITH_NONUI",
                        "AIRPOLLUTION_WITH_NONUI",
                        "VIETNAMTRIP_WITH_NONUI"
                        )

users_df = get_dataframe_from_db('select * from users;', DB_CONFIG)
users_df = users_df[users_df['crowdId'].isin(
    valid_user_crowdIds)]

for theme_ui_condition in THEME_UI_CONDITION:
    res = users_df.query('taskId == @theme_ui_condition')

# 問いかけ群ワーカー
QUESTION_UI_CONDITION = (
    "TPP_WITH_QUESTIONUI",
    "AIRPOLLUTION_WITH_QUESTIONUI",
    "VIETNAMTRIP_WITH_QUESTIONUI"
)

q_users_df = get_dataframe_from_db("select * from users;", QUESTION_DB_CONFIG)
q_users_df = q_users_df[q_users_df['crowdId'].isin(q_valid_user_crowdIds)]



for condition in QUESTION_UI_CONDITION:
    res = q_users_df.query('taskId == @condition')
    print(len(res))

question_UI_participant = [27, 32, 28]
perspective_UI_participant = [32, 30, 25]
non_UI_participant = [33, 35, 30]

participant_list = [
    question_UI_participant,
    perspective_UI_participant,
    non_UI_participant
]

participant_df = pd.DataFrame(participant_list)
participant_df.index = ['questionUI','perspectiveUI', 'nonUI']
participant_df.columns = ['TPP', '大気汚染', 'ベトナム旅行']
participant_df.head()





27
32
28


<ipython-input-4-afed83266c14>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
<ipython-input-4-afed83266c14>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,TPP,大気汚染,ベトナム旅行
questionUI,27,32,28
perspectiveUI,32,30,25
nonUI,33,35,30


In [7]:

# valid_user_crowdIds
# q_valid_user_crowdIds

def get_dataframe_without_invalid_user(df, valid_crowdIds):
  return df[df['crowdId'].isin(valid_crowdIds)]


## 意見文章の抽出

In [9]:
q_users_who_completed_all_task_df

# 大気汚染またはベトナム旅行がテーマだったワーカー
q_users_with_airPollution_or_vietnam_df = q_users_df[q_users_df['taskId'].isin(['AIRPOLLUTION_WITH_QUESTIONUI','VIETNAMTRIP_WITH_QUESTIONUI'])]
# 大気汚染またはベトナム旅行がテーマだったワーカーのcrowdId
q_crowdIds_with_airPollution_or_vietnam = q_users_with_airPollution_or_vietnam_df.crowdId.tolist()


q_opinion_document_df = get_dataframe_from_db('select * from opinionDocuments;', QUESTION_DB_CONFIG)
q_opinion_document_without_invalid_users = get_dataframe_without_invalid_user(
    q_opinion_document_df, q_crowdIds_with_airPollution_or_vietnam)
q_opinion_document_without_invalid_users.head()
q_opinion_document_without_invalid_users.to_csv('data/test.csv')
# q_opinion_document_list = q_opinion_document_without_invalid_users.content.tolist()

## 評価用シートの作成（csv化）
# csv_path = 'opinion.csv'

# with open(csv_path, "w", encoding="utf_8_sig") as f:
#     writer = csv.writer(f, lineterminator='\n')
#     for i in q_opinion_document_list:
#         writer.writerow([i])







<ipython-input-4-afed83266c14>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


## 集計結果

- 中央値
- 標準偏差

In [ ]:

AIRPOLLUTION_UI_CONDITION = (
    "AIRPOLLUTION_WITH_QUERYUI",
    "AIRPOLLUTION_WITH_NONUI",
)

VIETNAMTRIP_UI_CONDITION = (
    "VIETNAMTRIP_WITH_QUERYUI",
    "VIETNAMTRIP_WITH_NONUI"
)

QUESTION_UI_CONDITION = (
    "AIRPOLLUTION_WITH_QUESTIONUI",
    "VIETNAMTRIP_WITH_QUESTIONUI"
)

def get_target_column_median_and_std(df, target_column, condition_list):
    for condition in condition_list:
        target_df = df.query('taskId == @condition')
        median_value = target_df[target_column].median()
        std_value = target_df[target_column].std()
        print('=======')
        print(condition)
        print('中央値', median_value, '標準偏差', std_value)

# タスク所要時間
task_time_sql = """
select
	tt.crowdId, tt.restarted_at, tt.finished_at, users.taskId
from
	taskTime as tt
inner join
	users on tt.crowdId = users.crowdId
"""
# 問いかけ群以外
task_time_df = get_dataframe_from_db(task_time_sql, DB_CONFIG)
task_time_df_without_invalid_user = get_dataframe_without_invalid_user(
    task_time_df, valid_user_crowdIds)
task_time_df_without_invalid_user["taskTimeDuration"] = (
    task_time_df_without_invalid_user["finished_at"]-task_time_df_without_invalid_user["restarted_at"]).map(lambda x: x.total_seconds())
# 問いかけ群
q_task_time_df = get_dataframe_from_db(task_time_sql, QUESTION_DB_CONFIG)
q_task_time_df_without_invalid_user = get_dataframe_without_invalid_user(
    q_task_time_df, q_valid_user_crowdIds)
q_task_time_df_without_invalid_user["taskTimeDuration"] = (
    q_task_time_df_without_invalid_user["finished_at"]-q_task_time_df_without_invalid_user["restarted_at"]).map(lambda x: x.total_seconds())

# 大気汚染 && クエリ所要時間 && 問いかけ群以外
get_target_column_median_and_std(
    task_time_df_without_invalid_user, 'taskTimeDuration', AIRPOLLUTION_UI_CONDITION)
# ベトナム旅行 && クエリ所要時間 && 問いかけ群以外
get_target_column_median_and_std(
    task_time_df_without_invalid_user, 'taskTimeDuration', VIETNAMTRIP_UI_CONDITION)
# タスク所要時間 && 問いかけ群
get_target_column_median_and_std(
    q_task_time_df_without_invalid_user, 'taskTimeDuration', QUESTION_UI_CONDITION)


In [ ]:
# クエリ発行回数
query_count_sql = """
select
    users.crowdId, users.taskId, count(users.crowdId) as query_count
from
    queryLogs as ql
inner join
    users on ql.crowdId = users.crowdId
group by
    users.crowdId, users.taskId
"""

# 問いかけ群以外
query_count_df = get_dataframe_from_db(query_count_sql, DB_CONFIG)
query_count_df_without_invalid_user = get_dataframe_without_invalid_user(
    query_count_df, valid_user_crowdIds)

# 問いかけ群
q_query_count_df = get_dataframe_from_db(query_count_sql, QUESTION_DB_CONFIG)
q_query_count_df_without_invalid_user = get_dataframe_without_invalid_user(
    q_query_count_df, q_valid_user_crowdIds)

# 大気汚染 && クエリ発行回数 && 問いかけ群以外
get_target_column_median_and_std(
    query_count_df_without_invalid_user, 'query_count', AIRPOLLUTION_UI_CONDITION)
# 大気汚染 && クエリ発行回数 && 問いかけ群以外
get_target_column_median_and_std(
    query_count_df_without_invalid_user, 'query_count', VIETNAMTRIP_UI_CONDITION)
# クエリ発行回数 && 問いかけ群
get_target_column_median_and_std(
    q_query_count_df_without_invalid_user, 'query_count', QUESTION_UI_CONDITION)


In [ ]:

# 詳細ページ閲覧回数
visit_web_page_count_sql = """
select
	users.crowdId, users.taskId, count(users.crowdId) as visit_web_page_count
from
	visitPageLogs vl
inner join
	users on vl.crowdId = users.crowdId
group by
  users.crowdId, users.taskId;
"""

# 問いかけ群以外
visit_web_page_count_df = get_dataframe_from_db(visit_web_page_count_sql, DB_CONFIG)
visit_web_page_count_df_without_invalid_user = get_dataframe_without_invalid_user(
    visit_web_page_count_df, valid_user_crowdIds)
# 問いかけ群
q_visit_web_page_count_df = get_dataframe_from_db(
    visit_web_page_count_sql, QUESTION_DB_CONFIG)
q_visit_web_page_count_df_without_invalid_user = get_dataframe_without_invalid_user(
    q_visit_web_page_count_df, q_valid_user_crowdIds)

# 大気汚染 && 詳細ページ閲覧時間 && 問いかけ群以外
get_target_column_median_and_std(
    visit_web_page_count_df_without_invalid_user, 'visit_web_page_count', AIRPOLLUTION_UI_CONDITION)
# 大気汚染 && 詳細ページ閲覧時間 && 問いかけ群以外
get_target_column_median_and_std(
    visit_web_page_count_df_without_invalid_user, 'visit_web_page_count', VIETNAMTRIP_UI_CONDITION)
# 詳細ページ閲覧時間 && 問いかけ群
get_target_column_median_and_std(
    q_visit_web_page_count_df_without_invalid_user, 'visit_web_page_count', QUESTION_UI_CONDITION)



In [ ]:
# SERP閲覧時間
serp_time_duration_sql = """
select
	users.crowdId, users.taskId, st.time_on_serp
from
	serpTimeDuration as st
inner join
	users on st.crowdId = users.crowdId;
"""

# 問いかけ群以外
serp_time_duration_df = get_dataframe_from_db(
    serp_time_duration_sql, DB_CONFIG)
serp_time_duration_df_without_invalid_user = get_dataframe_without_invalid_user(
    serp_time_duration_df, valid_user_crowdIds)
# 問いかけ群
q_serp_time_duration_df = get_dataframe_from_db(
    serp_time_duration_sql, QUESTION_DB_CONFIG)
q_serp_time_duration_df_without_invalid_user = get_dataframe_without_invalid_user(
    q_serp_time_duration_df , q_valid_user_crowdIds)

# 大気汚染 && 詳細ページ閲覧時間 && 問いかけ群以外
get_target_column_median_and_std(
    serp_time_duration_df_without_invalid_user, 'time_on_serp', AIRPOLLUTION_UI_CONDITION)
# 大気汚染 && 詳細ページ閲覧時間 && 問いかけ群以外
get_target_column_median_and_std(
    serp_time_duration_df_without_invalid_user, 'time_on_serp', VIETNAMTRIP_UI_CONDITION)
# 詳細ページ閲覧時間 && 問いかけ群
get_target_column_median_and_std(
    q_serp_time_duration_df_without_invalid_user, 'time_on_serp', QUESTION_UI_CONDITION)

In [ ]:
# 最大クリック深度
max_ranking_sql = """
select
	vp.crowdId, max(ranking) as max_ranking, users.taskId
from
	visitPageLogs vp
inner join
	users on users.crowdId = vp.crowdId
group by
	crowdId, users.taskId;
"""

# 問いかけ群以外
max_ranking_df = get_dataframe_from_db(
    max_ranking_sql, DB_CONFIG)
max_ranking_df_without_invalid_user = get_dataframe_without_invalid_user(
    max_ranking_df, valid_user_crowdIds)
# 問いかけ群
q_max_ranking_df = get_dataframe_from_db(
    max_ranking_sql, QUESTION_DB_CONFIG)
q_max_ranking_df_without_invalid_user = get_dataframe_without_invalid_user(
    q_max_ranking_df, q_valid_user_crowdIds)

# 大気汚染 && 詳細ページ閲覧時間 && 問いかけ群以外
get_target_column_median_and_std(
    max_ranking_df_without_invalid_user, 'max_ranking', AIRPOLLUTION_UI_CONDITION)
# 大気汚染 && 詳細ページ閲覧時間 && 問いかけ群以外
get_target_column_median_and_std(
    max_ranking_df_without_invalid_user, 'max_ranking', VIETNAMTRIP_UI_CONDITION)
# 詳細ページ閲覧時間 && 問いかけ群
get_target_column_median_and_std(
    q_max_ranking_df_without_invalid_user, 'max_ranking', QUESTION_UI_CONDITION)

